In [ ]:
%matplotlib inline

In [ ]:
data_dir='/scr/ilz1/cbspython_surf/data/'

## Analysing surface data (if pretty pictures are not enough..)

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from vtk_rw import read_vtk, write_vtk
from plotting import plot_surf_stat_map

### Simplifying a high resolution mesh

Is a complex task, maybe not the right place to reinvent the wheel. There are good softwares to do this, like Meshlab (http://meshlab.sourceforge.net/)

**But Python can still help us**
* with changing between formats (e.g. vtk to ply https://github.com/juhuntenburg/brainsurfacescripts/blob/master/vtk2ply.py)
* with checking how good the algorithm perfomed / figuring out the best settings
* with mapping from one data to the other and from volumes to the surface
* ...


### Quality assessment of simplified mesh

https://github.com/juhuntenburg/brainsurfacescripts/blob/master/meshstats.py

In [ ]:
from meshstats import edges_areas_angles, meshstats

In [ ]:
# reading in the data
orig_file=data_dir + 'complex_mesh_rh.vtk'
simple_file = data_dir + 'simple_mesh_rh.vtk'
v_orig, f_orig, d_orig = read_vtk(orig_file)
v_simple, f_simple, d_simple = read_vtk(simple_file)

**NOTE: the simple mesh does not contain data!**

In [ ]:
# calculating edge lengths, face areas and angles and some stats on these measures
edges_orig, areas_orig, angles_orig =edges_areas_angles(v_orig, f_orig)
m_area_orig,s_area_orig,z_area_orig=meshstats(areas_orig)
m_edge_orig,s_edge_orig,z_edge_orig=meshstats(edges_orig)
m_angle_orig,s_angle_orig,z_angle_orig=meshstats(angles_orig)

In [ ]:
# same for simplified mesh
edges_simple, areas_simple, angles_simple=edges_areas_angles(v_simple, f_simple)
m_area_simple,s_area_simple,z_area_simple=meshstats(areas_simple)
m_edge_simple,s_edge_simple,z_edge_simple=meshstats(edges_simple)
m_angle_simple,s_angle_simple,z_angle_simple=meshstats(angles_simple)

In [ ]:
metrics = [[v_orig.shape[0], f_orig.shape[0], m_edge_orig, s_edge_orig, 
            m_area_orig, s_area_orig, m_angle_orig, s_angle_orig]]
metrics.append([v_simple.shape[0],f_simple.shape[0],m_edge_simple,s_edge_simple, 
                m_area_simple, s_area_simple, m_angle_simple, s_angle_simple])

metrics_df = pd.DataFrame(metrics, index=['orig', 'simple'], 
                          columns=['# vertex', '# face', 
                                   'mean edge', 'sdv edge', 
                                   'mean area', 'sdv area',
                                   'mean angle', 'sdv angle'])

metrics_df

In [ ]:
sns.set_context('poster')
fig = plt.figure(figsize=(20,5))
plt.suptitle('Edge length distribution', fontsize=20)
ax1=fig.add_subplot(121)
sns.distplot(edges_orig, hist=False, label='orig', color=sns.xkcd_rgb['medium green'])
sns.distplot(edges_simple, hist = False, label='simple', color=sns.cubehelix_palette(3)[2])
plt.xlabel('edge length [mm]')

ax2=fig.add_subplot(122)
sns.distplot(z_edge_orig, hist=False, label='orig', color=sns.xkcd_rgb['medium green'])
sns.distplot(z_edge_simple, hist = False, label='simple', color=sns.cubehelix_palette(3)[2])
plt.xlabel('edge length [z-score]')

In [ ]:
fig = plt.figure(figsize=(20,5))
plt.suptitle('Face area distribution', fontsize=20)
ax1=fig.add_subplot(121)
sns.distplot(areas_orig, hist=False, label='orig', color=sns.xkcd_rgb['medium green'])
sns.distplot(areas_simple, hist = False, label='simple', color=sns.cubehelix_palette(3)[2])
plt.xlabel('face area [mm^2]')

ax2=fig.add_subplot(122)
sns.distplot(z_area_orig, hist=False, label='orig', color=sns.xkcd_rgb['medium green'])
sns.distplot(z_area_simple, hist = False, label='simple', color=sns.cubehelix_palette(3)[2])
plt.xlabel('face area [z-score]')

In [ ]:
fig = plt.figure(figsize=(20,5))
plt.suptitle('Angle distribution', fontsize=20)
ax1=fig.add_subplot(121)
sns.distplot(angles_orig, hist=False, label='orig', color=sns.xkcd_rgb['medium green'])
sns.distplot(angles_simple, hist = False, label='simple', color=sns.cubehelix_palette(3)[2])
plt.xlabel('angles [radians]')

ax2=fig.add_subplot(122)
sns.distplot(z_angle_orig, hist=False, label='orig', color=sns.xkcd_rgb['medium green'])
sns.distplot(z_angle_simple, hist = False, label='simple', color=sns.cubehelix_palette(3)[2])
plt.xlabel('angles [z-score]')

### Mapping between high and low resolution mesh

In [ ]:
from scipy import spatial

In [ ]:
# find the vertices on the high resolution mesh, that are closest to the ones on the low resolution mesh
inaccuracy, mapping  = spatial.KDTree(v_orig).query(v_simple, k=1)

In [ ]:
# check inaccuracy
plt.hist(inaccuracy, bins=50);
plt.xlabel('inaccuray [mm]')

In [ ]:
# get the data at these vertices and write it on the low resolution mesh
data_simple = d_orig[mapping]
write_vtk('/tmp/simple_mesh_rh_data.vtk', v_simple, f_simple, data_simple)

Ideally, one would not want to sample only from the closest vertex on the highres mesh but from all vertices in the area. More on this here:
https://github.com/juhuntenburg/brainsurfacescripts/blob/master/simplification.py

### Sampling data from volume to surface

In [ ]:
from simplification import sample_volume, sample_simple

In [ ]:
# load volume data and parcellation

In [ ]:
# sample the volume data to the 

### Take a walk on the surface

In [ ]:
plot = plot_surf_stat_map(v_orig, f_orig, stat_map=np.squeeze(d_orig), elev=205, azim=0)